# Lab-07-1 Tips

- Reminder : Maximum Likelihood Estimation
- Reminder : Optimization via Gradient Descent
- Reminder : Overfitting and Regularization
- Training and Test Dataset
- Learning Rate
- Data Preprocessing

*(이론은 따로 필기)*

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Training and Test Dataset

In [6]:
# |x_train| = (m, 3), |y_train| = (m,)
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
# one-hot vector의 인덱스들을 m개 갖고 있음.

In [7]:
# |x_test| = (m', 3), |y_test| = (m',)
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [8]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)  # 3 => 3
    def forward(self, x):              # |x| = (m, 3) => (m, 3)
        return self.linear(x)

In [9]:
model = SoftmaxClassifierModel()

In [10]:
# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

## Training

In [12]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)   # |x_train| = (m, 3) => |prediction| = (m, 3)
        
        # cost 계산
        cost = F.cross_entropy(prediction, y_train)   # |y_train| = (m,)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()         # back propagation하고
        optimizer.step()        # gradient descent 수행
        
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

## Test (Validation)

In [14]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)                   # |x_test| = (m', 3) => |prediction| = (m', 3)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print("Accuracy: {}% Cost: {:.6f}".format(correct_count / len(y_test) * 100, cost.item()))

## Run

In [20]:
train(model, optimizer, x_train, y_train)
# x_train, y_train을 잘 설명하는 neural network의 parameter(model에 있음)가 optimizer를 통해 찾아지고 있는 것.
# parameter θ가 MLE를 통해서.

Epoch    0/20 Cost: 0.760971
Epoch    1/20 Cost: 0.758703
Epoch    2/20 Cost: 0.756465
Epoch    3/20 Cost: 0.754255
Epoch    4/20 Cost: 0.752074
Epoch    5/20 Cost: 0.749921
Epoch    6/20 Cost: 0.747795
Epoch    7/20 Cost: 0.745696
Epoch    8/20 Cost: 0.743623
Epoch    9/20 Cost: 0.741576
Epoch   10/20 Cost: 0.739554
Epoch   11/20 Cost: 0.737557
Epoch   12/20 Cost: 0.735584
Epoch   13/20 Cost: 0.733635
Epoch   14/20 Cost: 0.731709
Epoch   15/20 Cost: 0.729807
Epoch   16/20 Cost: 0.727927
Epoch   17/20 Cost: 0.726069
Epoch   18/20 Cost: 0.724233
Epoch   19/20 Cost: 0.722418


In [22]:
test(model, optimizer, x_test, y_test)
# loss 값이 높아졌다! -> test loss가 올라감. (overfitting)

Accuracy: 100.0% Cost: 0.160411


## Learning rate
learning rate이 너무 크면 diverge 하면서 cost가 점점 늘어난다 (**overshooting**).

$\theta \leftarrow \theta - \alpha \nabla_\theta L(x; \theta)$ ($\alpha$ : learning rate)

In [23]:
model = SoftmaxClassifierModel()

In [24]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [27]:
train(model, optimizer, x_train, y_train)
# 점점 커지고 있다!

Epoch    0/20 Cost: 200090.937500
Epoch    1/20 Cost: 1105673.750000
Epoch    2/20 Cost: 646816.500000
Epoch    3/20 Cost: 1064450.750000
Epoch    4/20 Cost: 1935385.125000
Epoch    5/20 Cost: 307693.093750
Epoch    6/20 Cost: 1494736.250000
Epoch    7/20 Cost: 668691.500000
Epoch    8/20 Cost: 661218.437500
Epoch    9/20 Cost: 541635.062500
Epoch   10/20 Cost: 1676443.125000
Epoch   11/20 Cost: 261552.109375
Epoch   12/20 Cost: 1099382.750000
Epoch   13/20 Cost: 818691.500000
Epoch   14/20 Cost: 1115072.625000
Epoch   15/20 Cost: 1478513.250000
Epoch   16/20 Cost: 738486.250000
Epoch   17/20 Cost: 827507.750000
Epoch   18/20 Cost: 643767.250000
Epoch   19/20 Cost: 135823.000000


learning rate이 너무 작으면 cost가 거의 줄어들지 않는다.

In [28]:
model = SoftmaxClassifierModel()

In [29]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [30]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [31]:
model = SoftmaxClassifierModel()

In [34]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)   # 0.1

In [33]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131977
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


## Data Preprocessing

데이터 전처리

In [38]:
# |x_train| = (m, 3), |y_train| = (m,)
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# regression 문제인 듯하다.

이럴 땐 min-square error 값을 쓴다. (MSE)

여기선 Standardization, 즉 정규분포로 만들어 주기. 우리가 가지고 있는 데이터가 정규분포를 따른다는 가정을 하고, mu, sigma 구해서 정규분포로 만들어 주기.

$x_j'=\frac{x_j - \mu_j}{\sigma_j}$

여기서 $\sigma$는 standard deviation, $\mu$는 평균값 이다.

$\sim N(0, 1)$

In [39]:
mu = x_train.mean(dim=0)

In [40]:
sigma = x_train.std(dim=0)

In [41]:
norm_x_train = (x_train - mu) / sigma

In [42]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


## Training with Preprocessed Data

In [43]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    def forward(self, x):
        return self.linear(x)

In [44]:
model = MultivariateLinearRegressionModel()

In [45]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [52]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # |x_train| = (m, 3), |prediction| = (m, 1)
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

In [53]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 0.301752
Epoch    1/20 Cost: 0.294306
Epoch    2/20 Cost: 0.287199
Epoch    3/20 Cost: 0.280397
Epoch    4/20 Cost: 0.273883
Epoch    5/20 Cost: 0.267629
Epoch    6/20 Cost: 0.261624
Epoch    7/20 Cost: 0.255850
Epoch    8/20 Cost: 0.250299
Epoch    9/20 Cost: 0.244960
Epoch   10/20 Cost: 0.239829
Epoch   11/20 Cost: 0.234892
Epoch   12/20 Cost: 0.230142
Epoch   13/20 Cost: 0.225577
Epoch   14/20 Cost: 0.221181
Epoch   15/20 Cost: 0.216950
Epoch   16/20 Cost: 0.212880
Epoch   17/20 Cost: 0.208971
Epoch   18/20 Cost: 0.205203
Epoch   19/20 Cost: 0.201581


왜 data preprocessing을 하는가?

```
y = [[999, 0.1], [1000, 0.2], [1010, 0.1], ...]
```
이런 식이면 학습에 있어 1번째 칼럼에만 집중할 것이기 때문.